# Good examples and tips in Edward

Some code that works and is useful, that I didn't want to throw away.

In [21]:
import numpy as np
import tensorflow as tf
import edward as ed
from pprint import pprint

In [5]:
# code from edward/examples/dirichlet_categorical.py:
# Inferring a categorical distribution with KLqp
tf.reset_default_graph()
sess = tf.InteractiveSession()
K = 4
N = 30
# DATA
pi_true = np.random.dirichlet(np.array([20.0, 30.0, 10.0, 10.0]))
z_data = np.array([np.random.choice(K, 1, p=pi_true)[0]
                   for n in range(N)])
print("pi: {}".format(pi_true))

# MODEL
pi = ed.models.Dirichlet(tf.ones(4))
z = ed.models.Categorical(probs=pi, sample_shape=N)

# INFERENCE
qpi = ed.models.Dirichlet(tf.nn.softplus(
    tf.get_variable("qpi/concentration", [K])))

inference = ed.KLqp({pi: qpi}, data={z: z_data})
inference.run(n_iter=1500, n_samples=30)

# sess = ed.get_session()
print("Inferred pi: {}".format(sess.run(qpi.mean())))

pi: [0.24454049 0.46987182 0.13792581 0.14766188]


/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


1500/1500 [100%] ██████████████████████████████ Elapsed: 6s | Loss: 43.773
Inferred pi: [0.29658943 0.33174667 0.22883385 0.14283012]


In [6]:
# example from examples/bayesian_linear_regression_sghmc.py
tf.reset_default_graph()
sess = tf.InteractiveSession()
def build_toy_dataset(N, noise_std=0.5):
    X = np.concatenate([np.linspace(0, 2, num=N / 2),
                        np.linspace(6, 8, num=N / 2)])
    y = 2.0 * X + 10 * np.random.normal(0, noise_std, size=N)
    X = X.astype(np.float32).reshape((N, 1))
    y = y.astype(np.float32)
    return X, y

N = 40  # number of data points
D = 1  # number of features
# DATA
X_train, y_train = build_toy_dataset(N)
X_test, y_test = build_toy_dataset(N)

# MODEL
X = tf.placeholder(tf.float32, [N, D])
w = ed.models.Normal(loc=tf.zeros(D), scale=tf.ones(D))
b = ed.models.Normal(loc=tf.zeros(1), scale=tf.ones(1))
y = ed.models.Normal(loc=ed.dot(X, w) + b, scale=tf.ones(N))

# INFERENCE
T = 5000                        # Number of samples.
nburn = 100                     # Number of burn-in samples.
stride = 10                     # Frequency with which to plot samples.
qw = ed.models.Empirical(params=tf.Variable(tf.random_normal([T, D])))
qb = ed.models.Empirical(params=tf.Variable(tf.random_normal([T, 1])))

inference = ed.SGHMC({w: qw, b: qb}, data={X: X_train, y: y_train})
inference.run(step_size=1e-3)

/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  """
/Users/jeromekafrouni/.pyenv/versions/3.6.1/envs/prob-prog/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  


5000/5000 [100%] ██████████████████████████████ Elapsed: 8s | Acceptance Rate: 1.000


### Building the Markov Chain:

In [11]:
# from issue https://github.com/blei-lab/edward/issues/450
from edward.models import Categorical, Dirichlet, Uniform, Mixture

chain_len = 30
n_hidden = 3
n_obs = 3

x_0 = Categorical(probs=tf.fill([n_hidden], 1.0 / n_hidden))

# transition matrix
T = tf.nn.softmax(tf.Variable(tf.random_uniform([n_hidden, n_hidden])), dim=0)

# emission matrix
E = tf.nn.softmax(tf.Variable(tf.random_uniform([n_obs, n_hidden])), dim=0)

# MODEL
x = []
y = []
for _ in range(chain_len):
    x_tm1 = x[-1] if x else x_0
    x_t = Categorical(probs=T[:, x_tm1])
    y_t = Categorical(probs=E[:, x_t])
    x.append(x_t)
    y.append(y_t)

In [17]:
# INFERENCE
qx = [Categorical(probs=tf.nn.softmax(tf.Variable(tf.ones(n_hidden))))
      for _ in range(chain_len)]

y_data = ([0] * 10) + ([1] * 10) + ([2] * 10)
y_data = map(np.array, y_data)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(T))

    inference = ed.KLqp(dict(zip(x, qx)), dict(zip(y, y_data)))
    inference.run(n_iter=20000)

    print(sess.run(T))
    print(sess.run(E))
    print(sess.run([foo.probs for foo in qx]))
    print(sess.run([foo.probs for foo in y]))

[[0.42720532 0.37529522 0.29069752]
 [0.20214134 0.34706113 0.3367309 ]
 [0.3706533  0.27764365 0.3725716 ]]
20000/20000 [100%] ██████████████████████████████ Elapsed: 107s | Loss: 10.563
[[0.8985785  0.03703917 0.03406724]
 [0.00348154 0.9607762  0.09740151]
 [0.09793993 0.00218464 0.8685312 ]]
[[9.9671930e-01 1.9515709e-03 2.1848876e-03]
 [1.7951096e-03 1.2189067e-03 9.9692792e-01]
 [1.4855342e-03 9.9682957e-01 8.8716898e-04]]
[array([0.9960776 , 0.0024412 , 0.00148117], dtype=float32), array([9.9728549e-01, 1.8385181e-03, 8.7593711e-04], dtype=float32), array([9.9807847e-01, 1.0312266e-03, 8.9030614e-04], dtype=float32), array([0.9966865 , 0.0017852 , 0.00152831], dtype=float32), array([0.9966016 , 0.0014918 , 0.00190652], dtype=float32), array([9.9790359e-01, 9.5059699e-04, 1.1458463e-03], dtype=float32), array([0.9974504 , 0.00139323, 0.0011563 ], dtype=float32), array([0.9974974 , 0.00111923, 0.00138342], dtype=float32), array([9.9841952e-01, 8.1004773e-04, 7.7040045e-04], dtype=

**Note:** this example seems to converge to something better that whay the guy said in the github example.

*Given my 3 hidden states, 3 observation types, and long changes of identical observations, I expect transition matrix to be close to diagonal and the emission matrix to look like a permutation matrix. I see non-converging loss info:*

Iteration     1 [  0%]: Loss = 35.123
Iteration  2000 [ 10%]: Loss = 46.565
Iteration  4000 [ 20%]: Loss = 47.923
Iteration  6000 [ 30%]: Loss = 56.581
Iteration  8000 [ 40%]: Loss = 53.431
Iteration 10000 [ 50%]: Loss = 49.397
Iteration 12000 [ 60%]: Loss = 55.551
Iteration 14000 [ 70%]: Loss = 47.537
Iteration 16000 [ 80%]: Loss = 47.690
Iteration 18000 [ 90%]: Loss = 47.107
Iteration 20000 [100%]: Loss = 38.514

*, non-uniform state probability distributions, and very uniform observation probabilities. Any idea what's going wrong in my setup or the solving of the problem?*

What data did we pass in:

In [23]:
y_data = ([0] * 10) + ([1] * 10) + ([2] * 10)
# for each categorical var y, he associated this matrix:
np.array(y_data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2])